In [1]:
import mbuild as mb
import numpy as np

from mbuild.prototypes.OH import OH

class COOH(mb.Compound):
    """Creates headgroup of a carboxylic acid"""
    def __init__(self, ester = None):
        super(COOH,self).__init__()
        if ester:
            self.add(mb.Particle(name='OE'), label='O[$]')
            self.add(mb.Particle(name='C'), label='C')
            self.add(mb.Particle(name='O', pos = [0, .123, 0]),label='O[$]')
            self.add_bond((self['C'],self['O'][1]))

            self.add(mb.Port(anchor=self['C'],
                orientation=[-1,-np.tan(50.5*np.pi/180),0],
                separation=.132/2), label='up')

            self['O'][0].translate([.15,0,0])
            theta = ((180-111) / 2) * np.pi / 180
            self['O'][0].rotate(-theta, around=[0,0,1])

            self.add_bond((self['C'],self['O'][0]))

            self.add(mb.Port(anchor=self['O'][0],
                orientation=[1,np.tan(52.5*np.pi/180),0],
                separation=.14/2), label='down')
        else:
            self.add(mb.Particle(name='O', pos = [0, .123, 0]),label='O[$]')
            self.add(mb.Particle(name='C'), label='C')
            self.add(OH(),label='OH')
            self.add_bond((self['C'],self['O'][0]))
            self.add(mb.Port(anchor=self['C'], 
                orientation=[1,-np.tan(32*np.pi/180), 0], 
                separation=.132/2),label='down')

            self.add(mb.Port(anchor=self['C'],
                orientation=[-1,-np.tan(34.5*np.pi/180),0],
                separation=.132/2), label='up')
            
            mb.force_overlap(move_this=self['OH'],
                    from_positions=self['OH']['up'],
                    to_positions=self['down'])


In [3]:
cooh = COOH(ester=True)
cooh.visualize(show_ports=True)

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


In [9]:
import mbuild as mb
import numpy as np

from mbuild.lib.moieties.ch3 import CH3
from mbuild.prototypes.alkyl_monomer import AlkylMonomer
from mbuild.prototypes.cooh import COOH

class FFA(mb.Compound):
    """Creates a saturated free fatty acid of n carbons based on user
    input"""
    def __init__(self, chain_length, hcap=None):
        super(FFA, self).__init__()
        
        if hcap:
            self.add(COOH(), label='head')
        else:
            self.add(COOH(ester=True), label='head')
        
        tail = mb.Polymer(AlkylMonomer(), (chain_length - 2))
        self.add(tail, label='tail')
        mb.x_axis_transform(self['tail'], new_origin=self['tail'][0],
                point_on_x_axis=self['tail'][6],
                point_on_xy_plane=self['tail'][3])
        self.add(CH3(), label='tailcap')
        
        self['head'].rotate(np.pi, [0,1,0])
        self['head'].translate([-self['tail'][3].pos[0],
            self['tail'][3].pos[1], 0])
        mb.force_overlap(move_this=self['tailcap'],
                from_positions=self['tailcap']['up'],
                to_positions=self['tail']['up'])
        
        self['head']['up'].spin(-np.pi/2, self['head']['C'].pos)
        mb.force_overlap(move_this=self['head'],
                from_positions=self['head']['up'],
                to_positions=self['tail']['down'])
        self.spin(np.pi/2, [0,1,0])

In [11]:
ffa = FFA(16, hcap=True)
ffa.visualize(show_ports=True)

/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.
